## Data Types

In [27]:
#give my computer a 5, it looks at it and thinks yup, i see thats an integer with value 5, and gives it back

In [18]:
#integer
5

5

In [29]:
#i give my computer a small math problem, and it gives back the answers

In [49]:
#float
8.3 + 5

13.3

In [ ]:
#explain that python is trying to keep the result the same type.

In [41]:
#example of python quirk (version 2 only)
10/3

3

In [21]:
#string
'string'

'string'

In [ ]:
string

In [22]:
'string' + 'another word'

'stringanother word'

In [50]:
'string' + 5

TypeError: cannot concatenate 'str' and 'int' objects

In [6]:
#boolean
5 == 5

True

In [7]:
'string' == 'another word'

False

In [23]:
#list

In [24]:
[1, 2, 3]

[1, 2, 3]

In [51]:
['one', 'two', 3]

['one', 'two', 3]

In [25]:
[4, 5 6]

SyntaxError: invalid syntax (<ipython-input-25-f9d004b92b31>, line 1)

## Variables -  store stuff for later

In [54]:
#notice that my computer isn't giving the input back to me right away. its storing it
variable1 = [4, 5, 6]

In [55]:
variable1[1]

5

In [56]:
variable2 = variable1

In [57]:
variable2

[4, 5, 6]

In [58]:
variable1[1] = 10

In [59]:
variable1

[4, 10, 6]

In [60]:
variable2

[4, 10, 6]

 ***

## Logic

In the programs we have seen till now, there has always been a series of statements faithfully executed by Python in exact top-down order. What if you wanted to change the flow of how it works or add program interpretation? In steps **control flow**.

### Conditionals

**`True`** and **`False`** are what we call booleans in Python and other programming languages. They are a special variable type with many potential uses; mainly they are used as a way to put a label on the truth of a statement. In Python, as most other languages, **`1`** and **`0`** also represent `Truthy` and `Falsy` values.

In [51]:
True == 1     # <- 1 is truthy

True

In [52]:
False == 0     # <- 0 is falsy

True

In addition, a wide variety of statements can evaluate to booleans. The ones that we will focus on today are the equalities, equal to and not equal to, and the inequalities, less than, greater than, less than or equal to and greater than or equal to. These comparisons are available in Python via **`==`** , **`!=`** , **`<`** , **`>`** , **`<=`** and **`>=`** , respectively.

In [54]:
5 == 4

False

In [55]:
5 != 4

True

In [56]:
5 < 4

False

In [57]:
5 > 4

True

In [58]:
5 <= 4

False

In [59]:
5 <= 5

True

In [60]:
5 >= 4

True

### Control Flow

The general syntax of an **`if`** statement in Python is:

```if condition:
    if_block_statement```

In [90]:
if Odds_Cardinals_Win_SB > 15/1:
    print("Just take my $20 til January then I want", Odds_Cardinals_Win_SB*20)

Just take my $20 til January then I want 320.0


Notice how the if statement ends in a colon **`:`**. This is the way that Python declares the start of an indentation block. 

![if control flow](https://camo.githubusercontent.com/3799b897439c900aedff9d17a931782d573f852d/687474703a2f2f7777772e7475746f7269616c73706f696e742e636f6d2f6370726f6772616d6d696e672f696d616765732f69665f73746174656d656e742e6a7067)

You can see that there are two branches created by the `if` statement, one when the condition is true, and the other when it is false. **But what if we wanted to check more than one thing** (i.e. have more than two branches in our flow diagram)?

In addition to the `if`, Python provides us with two other statements to build out those logical trees, the `elif` and the `else`. The `elif` is just like the `if` - it accepts a condition to check the truth of and has an indented code block that is executed when that condition evaluates to True. The `else` is similar, but it doesn't accept a condition. Instead, it mainly acts as a catch all for any other situation that you don't need to cover with your `ifs` and `elifs`. Note that there can only be a single `if` and up to a single `else`, but any number of `elifs` in an `if-elif-else` block.

```if condition:
    if_block_statement
elif condition:
    elif_block_statement
elif condition:
    another_elif_block_statement
else:
    else_block_statement```

***

The general syntax of a **`while`** loop in Python is:

```while condition:
    while_block_statement```

![while loop](https://camo.githubusercontent.com/06ad0bf93277549d18bc47eec9c71e8f36edfea4/687474703a2f2f7777772e70726f6772616d697a2e636f6d2f73697465732f7475746f7269616c3270726f6772616d2f66696c65732f435f7768696c655f6c6f6f702e6a7067)

**`while`** loops are an amazing tool which simply allow us to have a predefined chunk of code which we tell Python we want to run over and over under certain conditions.

So what are these conditions? They are in fact the conditions we learned about in the logic section (i.e. any expression that is evaluated to a boolean). So how does this work with `while`? Let's take a look at the structure of a `while` statement.

In [96]:
total= 0 
increment = 1
while increment <= 8:
    total = total + increment
    increment = increment + 1
    print("total = ", total)
    #print("increment = ", increment)
print(total)

total =  1
total =  3
total =  6
total =  10
total =  15
total =  21
total =  28
total =  36
36


Let's break down this code to see what is going on. On the first line, we declare a couple of variables, `total` and `increment`. `total` is the variable that we are going to aggregate our sum into, and `increment` is the first number that we start our adding at. The next line declares the start of our newly learned `while` block. It's condition is `increment <= 8`, and naturally reads as: **"while increment is less than or equal to 8", do stuff in the block.** The block then says we are to add the current value of `increment` to `total`, then add one to `increment`.

***

## Lists

What data structures does Python have for us to iterate through?

### Intro to Lists

From a high level, lists are collections of ordered items. These items can be of any type, and a list can contain items of different types (or all the same type). You can construct a list in one of two ways. The first is simply by passing an arbitrary number of items into square brackets, **`[]`**, separated by commas. The second is by passing an iterable into the **`list()`** constructor.

In [76]:
best_video_games_everrr = ["Street Fighter 2" , "Mario Bros.", "Super Metroid", "Tetris", "Super Mario Bros. 3"]

### List Operations

In [104]:
x = "tacocat"

In [117]:
x[::-1]

'tacocat'

In [114]:
best_video_games_everrr

['Street Fighter 2', 'Super Mario Bros. 3']

In [103]:
best_video_games_everrr[-2]

'Tetris'

In [121]:
best_video_games_everrr.append("Halo 2")

In [122]:
best_video_games_everrr.sort()

In [123]:
best_video_games_everrr

['Halo 2',
 'Halo 2',
 'Halo 2',
 'Mario Bros.',
 'Street Fighter 2',
 'Super Mario Bros. 3',
 'Super Metroid',
 'Tetris']

***

### Dictionaries

So far, the only collections that we have talked about are ordered. These are great as containers if there is some intrinsic order to the data that we're storing. However, there are plenty of times when we don't care about order, either because it simply doesn't matter or because the data are associated with each other in a different way. For example, say we have a bunch of state names and we want to associate each state's name with its capital. How would we do this in a list? One way would be to have tuples that store pairs of states and their capitals.

In [127]:
states_caps = {'Georgia': 'Atlanta', 'Colorado': 'Denver', 'Indiana': 'Indianapolis'}

In [128]:
states_caps['Colorado']

'Denver'